### Content 
- consistency checks for df_prods
- 3/4 mixed data types in df_ords_w
- missing values in df_ords_w
- handling duplicates in df_ords_w
- export clean data (products and orders)

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# importing data

path = r'/Users/lennartzeidler/Desktop/Uni/Instacart Basket Analysis 08.10.2022 (4)'

df_ords_w = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col=0)
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'))

### step 1 consistency checks for df_prods

In [3]:
# check for mixed types
for col in df_prods.columns.tolist():
  weird = (df_prods[[col]].applymap(type) != df_prods[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_prods[weird]) > 0:
    print (col)

product_name


### important 
It's interesting to note at this point that null values only appear in the column containing mixed data. We may eliminate null values from our data frame by changing the data type of the column (product name) to string. Meaning if we convert our the product_name column into strings the Nan values get converted into something different than null. ==> Since we want to eliminate the missing data, I choose not to change the data type.

In [4]:
# checking for null values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [5]:
# creating subset containing null values
# checking the null values by creating a subset for them using >> df_nan = df_prods[df_prods['product_name'].isnull() == True]
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]
df_prods_clean.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [6]:
# finding duplicates

df_dups = df_prods_clean[df_prods_clean.duplicated()] # ==> 5 duplicates were found

# delete duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates() 

In [7]:
df_prods_clean_no_dups.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [8]:
df_prods_clean_no_dups.shape 

(49672, 5)

### step 2

Three of the four columns are ID's what and there is nothing weird occuring. Only one thing is strange, thats the max price of, it is way to high(99999.000). I found out(through creating a subset of products where the price is higher than 25) that there are two products with a weird price. I now would ask a sales team member if those products have a that high price for a reason.  

products:  
Lowfat 2% Milkfat Cottage Cheese with a price of 14900.0  
2 % Reduced Fat Milk with a price of 99999.0  


comand for subset: t = df_prods[df_prods['prices'] > 25]

In [9]:
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,9.994136
std,14343.717401,38.316774,5.850282,453.519686
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,99999.000000


### step 3/4 mixed data types in df_ords_w

In [10]:
# check for mixed types
for col in df_ords_w.columns.tolist():
  weird = (df_ords_w[[col]].applymap(type) != df_ords_w[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords_w[weird]) > 0:
    print (col)
    
# no mixed data found

### step 5/6 missing values in df_ords_w

We have an unnamed column because, as I recently discovered, one column appears to have been imported in an odd manner. This column's value, which is simply a count of the table rows, can be deleted. 
I changed the import to resolve this problem. It appears that I exported the index column from our last task, which explains why it's being imported yet again. I simply instructed Python to use the first column as its index column.We need more storage space because we are storing the index column as well, which can be problematic with larger data sets and may also not be necessary.

In [11]:
# finding missing data

df_ords_w.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
order_day_of_week              0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

206209 missing values were found in the days_since_prior_order column.  


In [12]:
# creating subset containing null values

df_nan = df_ords_w[df_ords_w['days_since_prior_order'].isnull() == True]
df_nan.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
11,2168274,2,1,2,11,NaN
26,1374495,3,1,1,14,NaN
39,3343014,4,1,6,11,NaN
45,2717275,5,1,3,12,NaN


In [13]:
# checking counts  

df_nan['order_number'].value_counts()

1    206209
Name: order_number, dtype: int64

All orders with the order number of 1 have no days since prior order entry, because it's the customers first order.  That indicates that the order in question was the first one. It would not make sense to analyse data from the first order in this column, so we should always keep that in mind when analysing the days_since_prior_order column. We should use a df that excludes first orders when analysing this column.

### step 7/8 handling duplicates in df_ords_w

In [14]:
# finding duplicates

df_dups = df_ords_w[df_ords_w.duplicated()]

df_dups.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order


I did not find any duplicates.

### step 9 exporting data

In [19]:
# exporting cleaned orders df

df_ords_w.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_checked.cs.csv'), index=False)

In [22]:
# exporting cleaned products df

df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_checked.csv'), index=False)

In [24]:
df_ords_w.shape

(3421083, 6)